# IRIS - torch.nn
### M. Johnson, Feb 15, 2019

lr = 0.001  
batch size = 14

Acc: 0.92


In [2]:
import os
import torch
import torchvision
from torchvision import transforms
from my_classes import IrisDataset

# Data type for tensors
dtype = torch.float

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import my_classes as mc

##################
# IMPORTING DATA #
##################

iris_data = load_iris()
features, pre_labels = iris_data.data, iris_data.target

labels = mc.one_hot_encode(pre_labels, 3)

feature_train, feature_test, labels_train, labels_test = train_test_split(features, labels, random_state = 17)

# Load the standard scaler
sc = StandardScaler()

# Compute the mean and standard deviation based on the training data
sc.fit(feature_train)

# Scale the training data to be of mean 0 and of unit variance
feature_train = sc.transform(feature_train)

# Scale the test data to be of mean 0 and of unit variance
feature_test = sc.transform(feature_test)

####################

####################

import torch.utils.data as data_utils

batch_size = 14

# Training
train = data_utils.TensorDataset(torch.from_numpy(feature_train).float().to(device), 
                                 torch.from_numpy(labels_train).float().to(device))
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)
data_iter = iter(train_loader)
feats, labels = next(data_iter)

# Testing
test = data_utils.TensorDataset(torch.from_numpy(feature_test).float().to(device), 
                                 torch.from_numpy(labels_test).float().to(device))
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=True)
test_iter = iter(test_loader)

print(feats.shape)
print(labels.shape)

torch.Size([14, 4])
torch.Size([14, 3])


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(4, 8)
        self.fc2 = nn.Linear(8, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.softmax(self.fc3(x), dim=1)
        return x

    def name(self):
        return "IRIS relu h=8"

In [11]:
lr = 0.001

# Model
model = NeuralNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

data_iter = iter(train_loader)
iter_per_epoch = len(train_loader)
total_step = 5000

loss_list = []
loss_dict = {}

# Start training
for step in range(total_step):

    # Reset the data_iter
    if (step + 1) % iter_per_epoch == 0:
        data_iter = iter(train_loader)

    # Grab images and labels
    images, labels = next(data_iter)
    images.to(device)
    labels.to(device)
    
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, torch.max(labels,1)[1])

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_dict[step] = loss
    loss_list.append(loss)

    # Compute accuracy
    _, argmax = torch.max(outputs, 1)
    labels_ = torch.max(outputs, 1)[1]
    accuracy = (labels_ == argmax.squeeze()).float().mean()

    if (step + 1) % 100 == 0:
        print(f'Step [{step+1}/{total_step}], Loss: {loss.item():.4}, Acc: {accuracy.item():.2}')

Step [100/5000], Loss: 1.057, Acc: 1.0
Step [200/5000], Loss: 0.7763, Acc: 1.0
Step [300/5000], Loss: 0.6672, Acc: 1.0
Step [400/5000], Loss: 0.6345, Acc: 1.0
Step [500/5000], Loss: 0.6168, Acc: 1.0
Step [600/5000], Loss: 0.5228, Acc: 1.0
Step [700/5000], Loss: 0.4604, Acc: 1.0
Step [800/5000], Loss: 0.5084, Acc: 1.0
Step [900/5000], Loss: 0.4996, Acc: 1.0
Step [1000/5000], Loss: 0.3667, Acc: 1.0
Step [1100/5000], Loss: 0.5096, Acc: 1.0
Step [1200/5000], Loss: 0.56, Acc: 1.0
Step [1300/5000], Loss: 0.3704, Acc: 1.0
Step [1400/5000], Loss: 0.3796, Acc: 1.0
Step [1500/5000], Loss: 0.3998, Acc: 1.0
Step [1600/5000], Loss: 0.625, Acc: 1.0
Step [1700/5000], Loss: 0.3335, Acc: 1.0
Step [1800/5000], Loss: 0.2642, Acc: 1.0
Step [1900/5000], Loss: 0.3288, Acc: 1.0
Step [2000/5000], Loss: 0.4404, Acc: 1.0
Step [2100/5000], Loss: 0.7382, Acc: 1.0
Step [2200/5000], Loss: 0.2571, Acc: 1.0
Step [2300/5000], Loss: 0.5092, Acc: 1.0
Step [2400/5000], Loss: 0.4788, Acc: 1.0
Step [2500/5000], Loss: 0.344

In [25]:
# Test
model.eval()

test_iter = iter(test_loader)

correct, total = 0, 0

with torch.no_grad():
    for i in range(len(test_loader)):
        images, labels = next(test_iter)

        outputs = model(images)

        # Compute accuracy
        _, argmax = torch.max(outputs, 1)
        correct += (torch.max(labels, 1)[1] == argmax.squeeze()).float().sum()
        total += len(labels)
        
    
    acc = correct/total
    print(f'Acc: {acc:.2}')


Acc: 0.92
